In [1]:
import torch
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
from transformers import AutoProcessor, AutoModel
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, matthews_corrcoef

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "google/siglip-base-patch16-224"

processor = AutoProcessor.from_pretrained(model_name)

model = AutoModel.from_pretrained(model_name).to(device).eval()


In [3]:
df_allCarabids = pd.read_csv("/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/AllCarabids.csv", sep="\t")
df_insect1M = df_allCarabids[df_allCarabids["filepath"].str.startswith("Insect-1M")].copy()
df_insect1M.rename(columns={"filepath": "ImageFilePath", "class": "ScientificName"}, inplace=True)
cols = ['ImageFilePath', 'ScientificName']
df = df_insect1M[cols]
df['ImageFilePath'] = "/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/" + df['ImageFilePath']
df = df.dropna()
df.head(2)

/tmp/slurmtmp.831590/ipykernel_3027885/1676424945.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ImageFilePath'] = "/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/" + df['ImageFilePath']


,ImageFilePath,ScientificName
0,/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/Insect...,Cicindela sexguttata
1,/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/Insect...,Cicindela sexguttata


In [4]:
def extract_features(image_path) :
    
    image = Image.open(image_path).convert("RGB")
    
    inputs = processor(images=image, return_tensors="pt").to(device)
    
    with torch.no_grad():
        features = model.get_image_features(pixel_values=inputs["pixel_values"])
    
    return features.cpu().numpy()
    

In [5]:
X = np.vstack([extract_features(img) for img in tqdm(df["ImageFilePath"])])

le = LabelEncoder()

y = le.fit_transform(df["ScientificName"])

df_indices = df.index 

X_train, X_test, y_train, y_test, train_idx, test_idx = train_test_split(X, y, df_indices, test_size=0.2, random_state=42)

test_df = df.loc[test_idx].copy()

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test)


100%|██████████| 20278/20278 [26:14<00:00, 12.88it/s]


In [6]:
seed = 99

models = {
    "NaiveBayes": GaussianNB(),
    "LogisticRegression": LogisticRegression(max_iter=100),
    "NearestNeighbor": KNeighborsClassifier(n_neighbors=5), 
    "MLP_Baseline": MLPClassifier(hidden_layer_sizes=(100,), max_iter=200, random_state=seed)
}


predictions = {}

metrics = {}

for name, model in models.items():
    
    model.fit(X_train_scaled, y_train)
    preds = model.predict(X_test_scaled)
    predictions[name] = preds
    
    acc = accuracy_score(y_test, preds)
    prec = precision_score(y_test, preds, average="weighted")
    rec = recall_score(y_test, preds, average="weighted")
    f1 = f1_score(y_test, preds, average="weighted")
    bal_acc = balanced_accuracy_score(y_test, preds)
    mcc = matthews_corrcoef(y_test, preds)
    
    metrics[name] = {"Model": name, "Accuracy": acc, "Precision": prec, "Recall": rec, "F1-Score": f1, "Balanced Acc": bal_acc, "MCC": mcc}
    print(f"{name:<25} | Acc: {acc:.2%} | Prec: {prec:.2%} | Rec: {rec:.2%} | F1: {f1:.2%} | Bal Acc: {bal_acc:.2%} | MCC: {mcc:.4f}")


metrics_df = pd.DataFrame(metrics).T

/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:2480: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


NaiveBayes                | Acc: 30.33% | Prec: 31.83% | Rec: 30.33% | F1: 28.80% | Bal Acc: 17.78% | MCC: 0.3014


/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:2480: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


LogisticRegression        | Acc: 48.52% | Prec: 46.60% | Rec: 48.52% | F1: 45.27% | Bal Acc: 30.90% | MCC: 0.4835


/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:2480: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


NearestNeighbor           | Acc: 28.03% | Prec: 31.59% | Rec: 28.03% | F1: 27.31% | Bal Acc: 15.83% | MCC: 0.2782
MLP_Baseline              | Acc: 38.49% | Prec: 39.78% | Rec: 38.49% | F1: 36.86% | Bal Acc: 23.32% | MCC: 0.3831


/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:2480: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


In [7]:
test_df = test_df.assign(**{f"Pred_{name}": le.inverse_transform(pred) for name, pred in predictions.items()})
test_df.head(2)

,ImageFilePath,ScientificName,Pred_NaiveBayes,Pred_LogisticRegression,Pred_NearestNeighbor,Pred_MLP_Baseline
8222,/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/Insect...,Cicindela willistoni,Eunota circumpicta,Cicindela willistoni,Cicindela tranquebarica,Cicindela tranquebarica
15309,/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/Insect...,Cicindela hirticollis,Cicindela hirticollis,Cicindela repanda,Cicindela hirticollis,Cicindela hirticollis


In [8]:
metrics_df

,Model,Accuracy,Precision,Recall,F1-Score,Balanced Acc,MCC
NaiveBayes,NaiveBayes,0.303254,0.318346,0.303254,0.288036,0.177831,0.301393
LogisticRegression,LogisticRegression,0.485207,0.466045,0.485207,0.45267,0.308971,0.483485
NearestNeighbor,NearestNeighbor,0.280325,0.315929,0.280325,0.273126,0.158276,0.278157
MLP_Baseline,MLP_Baseline,0.384862,0.397818,0.384862,0.368582,0.233208,0.383091


In [9]:
test_df.to_csv("/users/PAS2136/rayees/CV4A - Benchmarking/Insect1M_dropna/10.SigLIP-linear-probing-species.csv", index=False)
metrics_df.to_csv("/users/PAS2136/rayees/CV4A - Benchmarking/Insect1M_dropna/10.SigLIP-linear-probing-species-metrics.csv", index=False)